In [1]:
%pip install torch
%pip install pymoo
%pip install pywin32

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os, sys

# Get the absolute path of the notebook's directory
notebook_dir = os.getcwd()

# Navigate to the parent directory
parent_dir = os.path.abspath(os.path.join(notebook_dir, ".."))

# Add the parent directory to sys.path so we can import modules
sys.path.append(parent_dir)

# Verify the path
print(f"Added to sys.path: {parent_dir}")

# Add it to sys.path
sys.path.append(parent_dir)

print(parent_dir)
from localityaware.module import *
from NSGA_nn.nsga import optimize_surr_nsga_1, optimize_surr_nsga_2
from FlashOperation.Refrig2DrumHeatExConstr1 import Refrig2DrumConstraintHeatExConstr

Added to sys.path: C:\Users\flin33\OneDrive - Johns Hopkins\ASPEN\Aspen
C:\Users\flin33\OneDrive - Johns Hopkins\ASPEN\Aspen
['C:\\Users\\flin33\\OneDrive - Johns Hopkins\\ASPEN\\Aspen\\NSGA_nn', 'C:\\ProgramData\\anaconda3\\python311.zip', 'C:\\ProgramData\\anaconda3\\DLLs', 'C:\\ProgramData\\anaconda3\\Lib', 'C:\\ProgramData\\anaconda3', '', 'C:\\ProgramData\\anaconda3\\Lib\\site-packages', 'C:\\ProgramData\\anaconda3\\Lib\\site-packages\\win32', 'C:\\ProgramData\\anaconda3\\Lib\\site-packages\\win32\\lib', 'C:\\ProgramData\\anaconda3\\Lib\\site-packages\\Pythonwin', 'C:\\Users\\flin33\\OneDrive - Johns Hopkins\\ASPEN\\Aspen', 'C:\\Users\\flin33\\OneDrive - Johns Hopkins\\ASPEN\\Aspen', 'C:\\Users\\flin33\\OneDrive - Johns Hopkins\\ASPEN\\Aspen']


In [3]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os

# Define the file path for saving/loading the data
data_file = "flash_simulation_data_sim_new.pkl"



print("Generating new data...")
assSim = Refrig2DrumConstraintHeatExConstr(AspenFile="../FlashOperation/FlashOperation_HeatExchanger.bkp",
                                        wdpath="../FlashOperation",
                                        visibility=True,
                                        Penalty=1e3)

Generating new data...
The current Directory is :  
C:\Users\flin33\OneDrive - Johns Hopkins\ASPEN\Aspen\NSGA_nn
The new Directory where you should also have your Aspen file is : 
C:\Users\flin33\OneDrive - Johns Hopkins\ASPEN\Aspen\FlashOperation
Aspen is active now. If you dont want to see aspen open again take VISIBITLY as False 

TEMPOUT: -65.0630049
The cost is 0.00
TEMPOUT: -65.3737009
The cost is 0.00
TEMPOUT: -63.4166894
The cost is 0.00
TEMPOUT: None
The cost is 0.00
TEMPOUT: None
The cost is 0.00
TEMPOUT: None
The cost is 0.00
TEMPOUT: None
The cost is 0.00
TEMPOUT: None
The cost is 0.00
TEMPOUT: None
The cost is 0.00
TEMPOUT: None
The cost is 0.00
TEMPOUT: -67.9285032
The cost is 0.00
TEMPOUT: None
The cost is 0.00
TEMPOUT: None
The cost is 0.00
TEMPOUT: -41.5834814
The cost is 0.00
TEMPOUT: -41.3230188
The cost is 0.00
TEMPOUT: -40.1724406
The cost is 0.00
TEMPOUT: -38.9136662
The cost is 0.00
TEMPOUT: -38.9107242
The cost is 0.00
TEMPOUT: -38.9782192
The cost is 0.00
TEMPO

In [ ]:
# Define parameter ranges
flash_1_range = np.linspace(1, 20, 10)
flash_2_range = np.linspace(1, 6, 10)

data_sim = []
for flash_1 in flash_1_range:
    for flash_2 in flash_2_range:
        print(flash_1, flash_2)
        x_unflat = assSim.unflatten_params([float(flash_1), float(flash_2)])
        data_sim.append([flash_1, flash_2, assSim.run_obj(x_unflat)])

data_sim = np.array(data_sim)

In [ ]:
data_file = "../FlashOperation/flash_simulation_data_nsga.pkl"

if os.path.exists(data_file):
    # Load existing data
    with open(data_file, "rb") as f:
        data_sim = pickle.load(f)
    print("Loaded existing data.")
else:
    # Create and save initial data
    with open(data_file, "wb") as f:
        pickle.dump(data_sim, f)
    print("Data file not found. Created and saved new data.")

In [ ]:
from torch.utils.data import Dataset

class DynamicDataset(Dataset):
    """Basic dataset for (x, y) pairs with dynamic adding"""
    def __init__(self, data):
        self.data = data  # expects a 2D numpy array: shape (n_samples, n_features + 1)

    def add_samples(self, new_data):
        self.data = np.vstack([self.data, new_data])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx, :-1]
        y = self.data[idx, -1]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

        
print(len(data_sim))

data_sim = np.array(data_sim)
x_data = data_sim[:, :2]
y_data = data_sim[:, 2]

x_min = x_data.min(axis=0)
x_max = x_data.max(axis=0)


scaler = TorchMinMaxScaler((-1, 1), 
                           max_vals=x_max,
                           min_vals=x_min,
                           min_y=0,
                           max_y=1e6 ,
                           scale_y=True)
#scale data_sim 
data_sim_xscaled, data_sim_yscaled = scaler.transform(data_sim[:, :2], data_sim[:, 2])

#recombine the data
data_sim_scaled = np.column_stack([data_sim_xscaled, data_sim_yscaled])

# **Initialize Model & Datasets**
dataset = DynamicDataset(data_sim_scaled)
model = MLP(2, [20,20,20,20,20], 1)

100
[[ 1.          1.          0.        ]
 [ 1.          3.11111111  0.        ]
 [ 1.          5.22222222  0.        ]
 [ 1.          7.33333333  0.        ]
 [ 1.          9.44444444  0.        ]
 [ 1.         11.55555556  0.        ]
 [ 1.         13.66666667  0.        ]
 [ 1.         15.77777778  0.        ]
 [ 1.         17.88888889  0.        ]
 [ 1.         20.          0.        ]
 [ 3.11111111  1.          0.        ]
 [ 3.11111111  3.11111111  0.        ]
 [ 3.11111111  5.22222222  0.        ]
 [ 3.11111111  7.33333333  0.        ]
 [ 3.11111111  9.44444444  0.        ]
 [ 3.11111111 11.55555556  0.        ]
 [ 3.11111111 13.66666667  0.        ]
 [ 3.11111111 15.77777778  0.        ]
 [ 3.11111111 17.88888889  0.        ]
 [ 3.11111111 20.          0.        ]
 [ 5.22222222  1.          0.        ]
 [ 5.22222222  3.11111111  0.        ]
 [ 5.22222222  5.22222222  0.        ]
 [ 5.22222222  7.33333333  0.        ]
 [ 5.22222222  9.44444444  0.        ]
 [ 5.22222222 11.5555

In [ ]:
from NSGA_nn.nsga import *
out = optimize_surr_nsga_2(model=model,
                   dataset=dataset,
                   assSim=assSim,
                   lr=1e-3, 
                   epoch=200,
                   min_vals=scaler.min_x,
                   max_vals=scaler.max_x,
                   scaler=scaler,
                   device='cpu',
                   iter=5,
                   print_loss=True,
                   print_it_data=True,
                   lambda_mse=1,
                   pop_size=100,
                   n_gen = 3,
                   new_data_size=5
                   )

Iteration 0: Training surrogate model...
device cpu
Epoch 0: Total Loss=0.9974
Epoch 10: Total Loss=0.9895
Epoch 20: Total Loss=0.9818
Epoch 30: Total Loss=0.9740
Epoch 40: Total Loss=0.9663
Epoch 50: Total Loss=0.9584
Epoch 60: Total Loss=0.9503
Epoch 70: Total Loss=0.9420
Epoch 80: Total Loss=0.9333
Epoch 90: Total Loss=0.9243
Epoch 100: Total Loss=0.9148
Epoch 110: Total Loss=0.9049
Epoch 120: Total Loss=0.8944
Epoch 130: Total Loss=0.8834
Epoch 140: Total Loss=0.8717
Epoch 150: Total Loss=0.8594
Epoch 160: Total Loss=0.8463
Epoch 170: Total Loss=0.8325
Epoch 180: Total Loss=0.8177
Epoch 190: Total Loss=0.8021
Epoch 200: Total Loss=0.7856
Epoch 210: Total Loss=0.7679
Epoch 220: Total Loss=0.7493
Epoch 230: Total Loss=0.7295
Epoch 240: Total Loss=0.7084
Epoch 250: Total Loss=0.6862
Epoch 260: Total Loss=0.6627
Epoch 270: Total Loss=0.6378
Epoch 280: Total Loss=0.6116
Epoch 290: Total Loss=0.5840
Epoch 300: Total Loss=0.5551
Epoch 310: Total Loss=0.5249
Epoch 320: Total Loss=0.4935
Ep

Epoch 80: Total Loss=0.0000
Epoch 90: Total Loss=0.0000
Epoch 100: Total Loss=0.0000
Epoch 110: Total Loss=0.0000
Epoch 120: Total Loss=0.0000
Epoch 130: Total Loss=0.0000
Epoch 140: Total Loss=0.0000
Epoch 150: Total Loss=0.0000
Epoch 160: Total Loss=0.0000
Epoch 170: Total Loss=0.0000
Epoch 180: Total Loss=0.0000
Epoch 190: Total Loss=0.0000
Epoch 200: Total Loss=0.0000
Epoch 210: Total Loss=0.0000
Epoch 220: Total Loss=0.0000
Epoch 230: Total Loss=0.0000
Epoch 240: Total Loss=0.0000
Epoch 250: Total Loss=0.0000
Epoch 260: Total Loss=0.0000
Epoch 270: Total Loss=0.0000
Epoch 280: Total Loss=0.0000
Epoch 290: Total Loss=0.0000
Epoch 300: Total Loss=0.0000
Epoch 310: Total Loss=0.0000
Epoch 320: Total Loss=0.0000
Epoch 330: Total Loss=0.0000
Epoch 340: Total Loss=0.0000
Epoch 350: Total Loss=0.0000
Epoch 360: Total Loss=0.0000
Epoch 370: Total Loss=0.0000
Epoch 380: Total Loss=0.0000
Epoch 390: Total Loss=0.0000
Epoch 400: Total Loss=0.0000
Epoch 410: Total Loss=0.0000
Epoch 420: Total

Epoch 70: Total Loss=0.0000
Epoch 80: Total Loss=0.0000
Epoch 90: Total Loss=0.0000
Epoch 100: Total Loss=0.0000
Epoch 110: Total Loss=0.0000
Epoch 120: Total Loss=0.0000
Epoch 130: Total Loss=0.0000
Epoch 140: Total Loss=0.0000
Epoch 150: Total Loss=0.0000
Epoch 160: Total Loss=0.0000
Epoch 170: Total Loss=0.0000
Epoch 180: Total Loss=0.0000
Epoch 190: Total Loss=0.0000
Epoch 200: Total Loss=0.0000
Epoch 210: Total Loss=0.0000
Epoch 220: Total Loss=0.0000
Epoch 230: Total Loss=0.0000
Epoch 240: Total Loss=0.0000
Epoch 250: Total Loss=0.0000
Epoch 260: Total Loss=0.0000
Epoch 270: Total Loss=0.0000
Epoch 280: Total Loss=0.0000
Epoch 290: Total Loss=0.0000
Epoch 300: Total Loss=0.0000
Epoch 310: Total Loss=0.0000
Epoch 320: Total Loss=0.0000
Epoch 330: Total Loss=0.0000
Epoch 340: Total Loss=0.0000
Epoch 350: Total Loss=0.0000
Epoch 360: Total Loss=0.0000
Epoch 370: Total Loss=0.0000
Epoch 380: Total Loss=0.0000
Epoch 390: Total Loss=0.0000
Epoch 400: Total Loss=0.0000
Epoch 410: Total 

Epoch 70: Total Loss=0.0000
Epoch 80: Total Loss=0.0000
Epoch 90: Total Loss=0.0000
Epoch 100: Total Loss=0.0000
Epoch 110: Total Loss=0.0000
Epoch 120: Total Loss=0.0000
Epoch 130: Total Loss=0.0000
Epoch 140: Total Loss=0.0000
Epoch 150: Total Loss=0.0000
Epoch 160: Total Loss=0.0000
Epoch 170: Total Loss=0.0000
Epoch 180: Total Loss=0.0000
Epoch 190: Total Loss=0.0000
Epoch 200: Total Loss=0.0000
Epoch 210: Total Loss=0.0000
Epoch 220: Total Loss=0.0000
Epoch 230: Total Loss=0.0000
Epoch 240: Total Loss=0.0000
Epoch 250: Total Loss=0.0000
Epoch 260: Total Loss=0.0000
Epoch 270: Total Loss=0.0000
Epoch 280: Total Loss=0.0000
Epoch 290: Total Loss=0.0000
Epoch 300: Total Loss=0.0000
Epoch 310: Total Loss=0.0000
Epoch 320: Total Loss=0.0000
Epoch 330: Total Loss=0.0000
Epoch 340: Total Loss=0.0000
Epoch 350: Total Loss=0.0000
Epoch 360: Total Loss=0.0000
Epoch 370: Total Loss=0.0000
Epoch 380: Total Loss=0.0000
Epoch 390: Total Loss=0.0000
Epoch 400: Total Loss=0.0000
Epoch 410: Total 

Epoch 40: Total Loss=0.0000
Epoch 50: Total Loss=0.0000
Epoch 60: Total Loss=0.0000
Epoch 70: Total Loss=0.0000
Epoch 80: Total Loss=0.0000
Epoch 90: Total Loss=0.0000
Epoch 100: Total Loss=0.0000
Epoch 110: Total Loss=0.0000
Epoch 120: Total Loss=0.0000
Epoch 130: Total Loss=0.0000
Epoch 140: Total Loss=0.0000
Epoch 150: Total Loss=0.0000
Epoch 160: Total Loss=0.0000
Epoch 170: Total Loss=0.0000
Epoch 180: Total Loss=0.0000
Epoch 190: Total Loss=0.0000
Epoch 200: Total Loss=0.0000
Epoch 210: Total Loss=0.0000
Epoch 220: Total Loss=0.0000
Epoch 230: Total Loss=0.0000
Epoch 240: Total Loss=0.0000
Epoch 250: Total Loss=0.0000
Epoch 260: Total Loss=0.0000
Epoch 270: Total Loss=0.0000
Epoch 280: Total Loss=0.0000
Epoch 290: Total Loss=0.0000
Epoch 300: Total Loss=0.0000
Epoch 310: Total Loss=0.0000
Epoch 320: Total Loss=0.0000
Epoch 330: Total Loss=0.0000
Epoch 340: Total Loss=0.0000
Epoch 350: Total Loss=0.0000
Epoch 360: Total Loss=0.0000
Epoch 370: Total Loss=0.0000
Epoch 380: Total Los